## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium

## Data Read-in

In [2]:
df = pd.read_csv('redfin_2023-04-10-09-32-59.csv')

## Data Clean

In [3]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [4]:
df = df.dropna(subset=['SOLD DATE'])

In [5]:
# Define list of desired months (excluding current month)
desired_months = ['March']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [6]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

False    2272
Name: PRICE, dtype: int64
-------
False    2091
True      181
Name: $/SQUARE FEET, dtype: int64
-------
False    2148
True      124
Name: YEAR BUILT, dtype: int64
-------


In [7]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [141]:
# df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
# df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
# df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
# df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
# df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [8]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1589,PAST SALE,March-31-2023,Single Family Residential,158 W Glen Ln,Riverdale,IL,60827.0,10000.0,3.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Riverdale/158-W-Glen...,MRED,11703480,N,Y,41.637650,-87.626297
293,PAST SALE,March-24-2023,Single Family Residential,4733 S Princeton Ave,Chicago,IL,60609.0,10000.0,4.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/4733-S-Princ...,MRED,11723346,N,Y,41.808124,-87.633074
1929,PAST SALE,March-24-2023,Single Family Residential,1430 W 115th St,Chicago,IL,60643.0,15000.0,4.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/1430-W-115th...,MRED,11666660,N,Y,41.685118,-87.657978
746,PAST SALE,March-8-2023,Single Family Residential,7404 S Dorchester Ave,Chicago,IL,60619.0,29900.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/7404-S-Dorch...,MRED,11711244,N,Y,41.760306,-87.591069
185,PAST SALE,March-17-2023,Single Family Residential,14238 Page Ave,Dixmoor,IL,60426.0,30000.0,3.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Dixmoor/14238-Page-A...,MRED,11715997,N,Y,41.630250,-87.659775
1998,PAST SALE,March-17-2023,Single Family Residential,15423 Winchester Ave,Harvey,IL,60426.0,34000.0,3.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Harvey/15423-Winches...,MRED,11662613,N,Y,41.609054,-87.664126
1276,PAST SALE,March-10-2023,Single Family Residential,2444 S Keeler Ave,Chicago,IL,60623.0,34900.0,4.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/2444-S-Keele...,MRED,11702040,N,Y,41.846536,-87.729719
1625,PAST SALE,March-28-2023,Single Family Residential,13924 S Edbrooke Ave,Riverdale,IL,60827.0,35000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Riverdale/13924-S-Ed...,MRED,11698665,N,Y,41.641576,-87.618696
1635,PAST SALE,March-7-2023,Single Family Residential,8001 S Laflin St,Chicago,IL,60620.0,35000.0,3.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/8001-S-Lafli...,MRED,11682349,N,Y,41.748440,-87.660675
638,PAST SALE,March-17-2023,Single Family Residential,12100 S Normal Ave,Chicago,IL,60628.0,36500.0,3.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/12100-S-Norm...,MRED,11718747,N,Y,41.674076,-87.634846


In [10]:
print(df_filtered['URL'].iloc[293])

https://www.redfin.com/IL/Chicago/4733-S-Princeton-Ave-60609/home/13952886


In [144]:
# # Correct the prices, if needed
# df_filtered.at[618,'PRICE']=(295000)
# df_filtered.at[832,'PRICE']=(355000)

In [11]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [146]:
# # Corrections, if needed
# df_filtered.at[1279,'$/SQUARE FEET']=(395000/1316)
# df_filtered.at[402,'$/SQUARE FEET']=(220000/1244)

In [12]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['ADDRESS','CITY','$/SQUARE FEET']]

,ADDRESS,CITY,$/SQUARE FEET
293,4733 S Princeton Ave,Chicago,7.0
1589,158 W Glen Ln,Riverdale,11.0
1929,1430 W 115th St,Chicago,16.0
1917,15433 Myrtle Ave,Harvey,19.0
1276,2444 S Keeler Ave,Chicago,22.0
2198,7215 S Princeton Ave,Chicago,23.0
1044,315 Wildwood Dr,Park Forest,23.0
1635,8001 S Laflin St,Chicago,23.0
746,7404 S Dorchester Ave,Chicago,24.0
201,137 W 104th Pl,Chicago,30.0


In [13]:
print(df_filtered.URL.iloc[217])

https://www.redfin.com/IL/Chicago/2123-W-54th-Pl-60609/home/13956133


In [149]:
# Corrections, if needed
#
#
#

In [150]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [14]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [15]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [16]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [17]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [18]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [156]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"March 2023 Home Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map

In [157]:
m.save('index.html')

## Summary Info

In [158]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [159]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [160]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [161]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
Lincoln Park 2550, 2550 N Lakeview Ave Unit S801 Chicago | Price $4,400,000 | $1,110 psf | Year built: 2012
Least Expensive
CHI - West Garfield Park, 3806 W Washington Blvd Unit DG Chicago | Price $25,000 | $36 psf | Year built: nan
Highest Price Per Square Foot
CHI - Near North Side, 14 W Superior St #5304 Chicago | Price $3,775,000 | $1,258 psf | Year built: 2022
Lowest Price Per Square Foot
CHI - South Shore, 6901 S Oglesby Ave Unit 10C Chicago | Price $70,000 | $27 psf | Year built: 1929
Newest
CHI - Lincoln Park, 1823 N Sedgwick St Unit PHS Chicago | Price $2,550,000 | $nan psf | Year built: 2023
Oldest
CHI - Near West Side, 1000 W Washington Blvd #301 Chicago | Price $905,000 | $393 psf | Year built: 1882


## Time on Market Calculator

In [162]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [163]:
from datetime import datetime, timedelta

date1 = datetime(2022, 12, 15) ## List (Earlier) date
date2 = datetime(2023, 3, 30) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

105


## Map URL Snagger

In [164]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [165]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CookCounty_condo_sales_month_ending_march_2023


## Get Summary Data

In [166]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 1784
--------
Total sale price: $663,220,457
--------
Median sale price: $275,000
--------
Max sale price: $4,400,000
--------
Min sale price: $25,000
------------------------------------------------
PSF INFO
Max price per square foot: $1,258
--------
Min price per square foot: $27
--------
Median price per square foot: $231
------------------------------------------------
CONDO AGES
Newest building: 2023.0
----------
Oldest building: 1882.0
----------
Average building age: 1977.679291386683


## To CSV

In [167]:
df_filtered.to_csv(f'{desired_months}_condo.csv')